<a href="https://colab.research.google.com/github/souzamichel/ml_trading_test_code/blob/main/ml_trading_backtest_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# 0. Instala dependências se necessário
import sys
import subprocess

def install_if_missing(pkg):
    try:
        __import__(pkg)
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])

for pkg in ("yfinance", "optuna"):
    install_if_missing(pkg)

In [13]:
# 1. Imports
import os
import csv
import logging
import numpy as np
import pandas as pd
import yfinance as yf
import optuna
from joblib import Parallel, delayed
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import TimeSeriesSplit
from optuna.exceptions import TrialPruned

In [14]:
# 2. Configuração de logs
LOG_FILENAME = "optimization.log"
CSV_FILENAME = "trial_results.csv"

logger = logging.getLogger("optuna_rf")
logger.setLevel(logging.INFO)

# File handler
fh = logging.FileHandler(LOG_FILENAME)
fh.setLevel(logging.INFO)
fh.setFormatter(logging.Formatter(
    "%(asctime)s %(levelname)s [Trial %(trial_number)s] %(message)s"
))
logger.addHandler(fh)

# Console handler
ch = logging.StreamHandler(sys.stdout)
ch.setLevel(logging.INFO)
ch.setFormatter(logging.Formatter(
    "%(asctime)s %(levelname)s [Trial %(trial_number)s] %(message)s"
))
logger.addHandler(ch)

In [15]:
# 3. Cria CSV de auditoria (header)
if not os.path.exists(CSV_FILENAME):
    with open(CSV_FILENAME, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([
            "trial_number",
            "ma_short", "ma_long", "rsi_period",
            "n_estimators", "max_depth",
            "avg_recovery", "avg_win_rate",
            "state"
        ])

In [16]:
# 4. Indicadores e métricas
def compute_rsi(series: pd.Series, period: int = 14) -> pd.Series:
    delta    = series.diff()
    gain     = delta.clip(lower=0)
    loss     = -delta.clip(upper=0)
    avg_gain = gain.rolling(period).mean()
    avg_loss = loss.rolling(period).mean()
    rs       = avg_gain / avg_loss
    return 100 - (100 / (1 + rs))

def calculate_metrics(returns: pd.Series) -> tuple:
    cumulative  = (1 + returns).cumprod() - 1
    running_max = cumulative.cummax()
    drawdown    = cumulative - running_max
    max_dd      = drawdown.min()
    total_ret   = cumulative.iloc[-1]
    win_rate    = (returns > 0).sum() / len(returns) if len(returns) else 0.0
    recovery    = (total_ret / abs(max_dd)) if max_dd < 0 else np.nan
    return recovery, win_rate

In [17]:
# 5. Carrega dados históricos
symbol = "AAPL"
data = yf.download(symbol, period="5y", interval="1d",
                   auto_adjust=True, progress=False)
data["Return"] = data["Close"].pct_change()
data.dropna(inplace=True)

In [18]:

# 6. Função-objetivo com pruning e logs
def objective(trial: optuna.trial.Trial) -> float:
    # Espaço de busca
    ma_short     = trial.suggest_int("ma_short", 5, 15, step=5)
    ma_long      = trial.suggest_int("ma_long", 10, 30, step=10)
    if ma_long <= ma_short:
        ma_long = ma_short + 5
    rsi_period   = trial.suggest_categorical("rsi_period", [14, 21])
    n_estimators = trial.suggest_categorical("n_estimators", [50, 100])
    max_depth    = trial.suggest_categorical("max_depth", [None, 5, 10])

    # Contexto para logs
    logger = logging.LoggerAdapter(logging.getLogger("optuna_rf"),
                                   {"trial_number": trial.number})
    params = {
        "ma_short": ma_short, "ma_long": ma_long, "rsi_period": rsi_period,
        "n_estimators": n_estimators, "max_depth": max_depth
    }
    logger.info(f"Starting trial with params {params}")

    # Constroi DataFrame de features e target
    df = data.copy()
    df["MA_short"] = df["Close"].rolling(ma_short).mean()
    df["MA_long"]  = df["Close"].rolling(ma_long).mean()
    df["RSI"]      = compute_rsi(df["Close"], rsi_period)
    df.dropna(inplace=True)

    fut = df["Close"].pct_change().shift(-1)
    df["Target"] = (fut > 0).astype(int)
    df.dropna(inplace=True)

    X   = df[["MA_short", "MA_long", "RSI"]]
    y   = df["Target"]
    ret = df["Close"].pct_change().loc[X.index]

    tscv = TimeSeriesSplit(n_splits=5)

    # Avalia folds em paralelo com pruning
    def eval_fold(train_idx, test_idx, fold_idx):
        X_tr, X_te = X.iloc[train_idx], X.iloc[test_idx]
        y_tr = y.iloc[train_idx]
        ret_te = ret.iloc[test_idx]

        if y_tr.nunique() < 2 or len(X_te) < 2:
            return np.nan, np.nan

        model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            random_state=42
        )
        model.fit(X_tr, y_tr)
        preds = model.predict(X_te)

        df_sig        = pd.DataFrame({"signal": preds}, index=X_te.index)
        df_sig["ret"] = ret_te
        df_sig["strat"] = df_sig["signal"].shift(1, fill_value=0) * df_sig["ret"]

        if df_sig["strat"].empty:
            return np.nan, np.nan

        recovery, win_rate = calculate_metrics(df_sig["strat"])
        # report & prune
        trial.report(recovery, step=fold_idx)
        logger.info(f"Fold {fold_idx} — recovery: {recovery:.4f}, win_rate: {win_rate:.2%}")
        if trial.should_prune():
            logger.info(f"Pruning trial at fold {fold_idx}")
            raise TrialPruned()
        return recovery, win_rate

    # Executa os folds
    try:
        folds = Parallel(n_jobs=-1)(
            delayed(eval_fold)(tr, te, idx + 1)
            for idx, (tr, te) in enumerate(tscv.split(X))
        )
    except TrialPruned:
        # Grava no CSV o estado pruneado
        with open(CSV_FILENAME, "a", newline="") as f:
            writer = csv.writer(f)
            writer.writerow([
                trial.number,
                ma_short, ma_long, rsi_period,
                n_estimators, max_depth,
                np.nan, np.nan,
                "PRUNED"
            ])
        raise

    # Processa resultados completos
    recovs, wins = zip(*folds)
    recovs = np.array(recovs, dtype=float)
    recovs = recovs[~np.isnan(recovs)]

    # Se algum fold inválido, retorna zero
    if len(recovs) < tscv.get_n_splits():
        mean_recov, mean_win = 0.0, 0.0
        state = "FAILED"
    else:
        mean_recov, mean_win = recovs.mean(), np.array(wins).mean()
        state = "COMPLETE"

    # Persiste auditoria no CSV
    with open(CSV_FILENAME, "a", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([
            trial.number,
            ma_short, ma_long, rsi_period,
            n_estimators, max_depth,
            f"{mean_recov:.6f}", f"{mean_win:.6f}",
            state
        ])

    logger.info(f"Finished trial — avg_recovery: {mean_recov:.4f}, avg_win_rate: {mean_win:.2%}")
    return mean_recov

In [19]:
# 7. Execução do estudo com pruner
pruner = optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=2)
study = optuna.create_study(direction="maximize", pruner=pruner)
study.optimize(objective, n_trials=50, n_jobs=1)

[I 2025-07-27 21:21:45,827] A new study created in memory with name: no-name-99b7065a-2881-49bf-9fc0-6ccddf3b27e8


2025-07-27 21:21:45,831 INFO [Trial 0] Starting trial with params {'ma_short': 15, 'ma_long': 20, 'rsi_period': 21, 'n_estimators': 50, 'max_depth': 5}


INFO:optuna_rf:Starting trial with params {'ma_short': 15, 'ma_long': 20, 'rsi_period': 21, 'n_estimators': 50, 'max_depth': 5}


2025-07-27 21:21:47,494 INFO [Trial 0] Finished trial — avg_recovery: 0.0000, avg_win_rate: 0.00%


INFO:optuna_rf:Finished trial — avg_recovery: 0.0000, avg_win_rate: 0.00%
[I 2025-07-27 21:21:47,496] Trial 0 finished with value: 0.0 and parameters: {'ma_short': 15, 'ma_long': 10, 'rsi_period': 21, 'n_estimators': 50, 'max_depth': 5}. Best is trial 0 with value: 0.0.


...

INFO:optuna_rf:Finished trial — avg_recovery: 0.0000, avg_win_rate: 0.00%
[I 2025-07-27 21:22:09,745] Trial 48 finished with value: 0.0 and parameters: {'ma_short': 10, 'ma_long': 20, 'rsi_period': 21, 'n_estimators': 50, 'max_depth': None}. Best is trial 5 with value: 0.7396212730710559.


2025-07-27 21:22:09,758 INFO [Trial 49] Starting trial with params {'ma_short': 15, 'ma_long': 30, 'rsi_period': 21, 'n_estimators': 50, 'max_depth': None}


INFO:optuna_rf:Starting trial with params {'ma_short': 15, 'ma_long': 30, 'rsi_period': 21, 'n_estimators': 50, 'max_depth': None}


2025-07-27 21:22:10,122 INFO [Trial 49] Finished trial — avg_recovery: -0.2274, avg_win_rate: 17.06%


INFO:optuna_rf:Finished trial — avg_recovery: -0.2274, avg_win_rate: 17.06%
[I 2025-07-27 21:22:10,124] Trial 49 finished with value: -0.22736604764372803 and parameters: {'ma_short': 15, 'ma_long': 30, 'rsi_period': 21, 'n_estimators': 50, 'max_depth': None}. Best is trial 5 with value: 0.7396212730710559.


In [20]:

# 8. Resultados finais
print("\nBest trial:")
best = study.best_trial
print(f"  Trial #: {best.number}")
print(f"  Params: {best.params}")
print(f"  Avg WF Recovery: {best.value:.4f}")
print(f"\nVeja '{LOG_FILENAME}' e '{CSV_FILENAME}' para logs e auditoria detalhada.")


Best trial:
  Trial #: 5
  Params: {'ma_short': 15, 'ma_long': 10, 'rsi_period': 21, 'n_estimators': 50, 'max_depth': None}
  Avg WF Recovery: 0.7396

Veja 'optimization.log' e 'trial_results.csv' para logs e auditoria detalhada.
